In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
x_npz = np.load("flowers.npz")
x_data = x_npz['arr_0']
y_npz = np.load("labels.npz")
y_true = y_npz['arr_0']

In [ ]:
array = []
for i in range(4323):
    c = x_data[i].astype(np.float)
    b = c.ravel()
    array.append(b)
# xpd = pd.DataFrame(data=array)
# ypd = pd.DataFrame(data=y_true,columns=['y'])
data = pd.concat([(pd.DataFrame(data=array)),pd.DataFrame(data=y_true,columns=['labels'])],axis=1)

In [ ]:
image_data = x_data[:,1:]

In [ ]:
trainLabelCounts = data['labels'].value_counts(sort = False)
trainLabelCounts

In [ ]:
def show_img(img):
    img = img.reshape(128,128)
    plt.imshow(img)

show_img(x_data[1])

In [ ]:
data = data.reindex(np.random.permutation(data.index))

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(array, y_true, test_size=0.07, random_state=42)

In [ ]:
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [ ]:
#one hot decoding
def one_hot_encode(labels,classes=5):
    one_hot = np.zeros([len(labels),6])
    for i in range(len(labels)):
        one_hot[i,labels[i]] = 1
    return one_hot
train_labels = one_hot_encode(y_train)
test_labels = one_hot_encode(y_test)

In [ ]:
hm_epochs = 10
batch_size = 100
n_classes = 6
n_nodes_h1 = 500
n_nodes_h2 = 500
n_nodes_h3 = 500

In [ ]:
x = tf.placeholder(tf.float32,[None,16384])
y = tf.placeholder(tf.float32)

In [ ]:
def neural_network_modal(data):
    
    hidden_layer1 = { 'weights':tf.Variable(tf.random_normal([16384,n_nodes_h1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_h1])) }
    
    hidden_layer2 = { 'weights':tf.Variable(tf.random_normal([n_nodes_h1,n_nodes_h2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_h2])) }
    
    hidden_layer3 = { 'weights':tf.Variable(tf.random_normal([n_nodes_h2,n_nodes_h3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_h3])) }
    
    output_layer = {  'weights':tf.Variable(tf.random_normal([n_nodes_h3,n_classes])),
                      'biases':tf.Variable(tf.random_normal([n_classes])) }
    
    l1 = tf.add(tf.matmul(data,hidden_layer1['weights']),hidden_layer1['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1,hidden_layer2['weights']),hidden_layer2['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2,hidden_layer3['weights']),hidden_layer3['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.matmul(l3,output_layer['weights']),output_layer['biases'])
    
    return output    

In [ ]:
def training_model(x):
    prediction = neural_network_modal(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(20):
            epoch_loss = 0
            batch_start = 1
            batch_end = batch_start + batch_size
            for _ in range(22):
                _,c = sess.run([optimizer,cost],feed_dict = {x:x_train[batch_start:batch_end],y:train_labels[batch_start:batch_end]})
                epoch_loss = epoch_loss +c
                batch_start = batch_end+1
                batch_end = batch_end + batch_size
                
            print("Epoch",epoch, "completed out of ",hm_epochs,"with loss:",epoch_loss)
        
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        
        
        print('Accuracy:',accuracy.eval({x:x_test, y:test_labels}))      
        
        
        
training_model(x)    